In [1]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv('../../data/ST001937_AN003150.csv')

Dropping not needed columns and renaming the Phenotypes column to labels

In [2]:
df = df.drop(['Sample ID', 'RAW_FILE_NAME'], axis=1)
df['Phenotypes'] = df['Phenotypes'].apply(lambda x: 0 if x == 'Healthy Controls' else 1 if x == 'Maligant SPNS' else 2)

Filling missing values with the mean

In [3]:
df.fillna(df.mean(), inplace=True)

Performing SVM RBF classification and plotting the results

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import numpy as np
import seaborn as sns

# Splitting the data into features (X) and labels (y)
X = df.drop('Phenotypes', axis=1)
y = df['Phenotypes']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Lista di valori gamma da testare da 0.001 a 1 con passo di 0.001
gamma_values = np.arange(0.0001, 0.011, 0.00001)

# Liste per memorizzare gli auc e confusion matrices corrispondenti
auc = []
conf_matrices = []

# Iterazione su diversi valori di gamma
for gamma in gamma_values:
    # Performing SVM RBF classification
    svm_classifier = SVC(kernel='rbf', C=13.2, gamma=gamma, random_state=42)
    svm_classifier.fit(X_train, y_train)

    # Predicting the labels for the test set
    y_pred = svm_classifier.predict(X_test)

    # Evaluating the model
    auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Aggiunta delle metriche alle liste
    auc.append(auc)
    conf_matrices.append(conf_matrix)

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
from scipy.interpolate import make_interp_spline

# Plotting the accuracy results
plt.figure(figsize=(10, 6))


plt.plot(gamma_values, auc)

plt.title('Accuracy vs. Gamma for SVM RBF')
plt.xlabel('Gamma')
plt.ylabel('Accuracy')
# Flagging the best accuracy value with a vertical line and a dot on the curve and the corresponding gamma value and accuracy
plt.axvline(gamma_values[np.argmax(auc)], color='r', linestyle='--')
plt.plot(gamma_values[np.argmax(auc)], np.max(auc), 'ro')
plt.show()

Visualizzazione delle confusion matrices per il best gamma

In [ ]:
best_gamma = gamma_values[np.argmax(auc)]
best_conf_matrix = conf_matrices[np.argmax(auc)]
plt.figure(figsize=(10, 6))
sns.heatmap(conf_matrix, annot=True, fmt="g", cmap="Blues", xticklabels=['Healthy', 'Malignant', 'Benign'], yticklabels=['Healthy', 'Malignant', 'Benign'])
plt.title('Confusion Matrix for SVM RBF (Gamma = {})'.format(best_gamma))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

print('Accuracy: {}'.format(np.max(auc)))